In [ ]:
import torch
import shap
import numpy as np
from lime.lime_text import LimeTextExplainer
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_path = r"E:\DS+ML\AIM3\Week5\fine-tuned-model"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

sample_text = "በአዲስ አበባ ውስጥ አዲስ ስልክ በ 5000 ብር ይገኛል።"

def predict(masked_inputs):
    tokenized_inputs = tokenizer(masked_inputs, return_tensors="pt", padding=True, truncation=True)
    input_ids = tokenized_inputs["input_ids"]
    attention_mask = tokenized_inputs["attention_mask"]
    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
    probabilities = torch.softmax(logits, dim=-1).numpy()
    return probabilities.astype(np.float32)

masker = shap.maskers.Text(tokenizer)
explainer = shap.Explainer(predict, masker)
shap_values = explainer([sample_text])

print("SHAP Analysis:")
shap.plots.text(shap_values)

def predict_proba(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.softmax(logits, dim=-1).numpy()
    return probabilities.astype(np.float32)

explainer_lime = LimeTextExplainer(class_names=model.config.id2label.values())
exp = explainer_lime.explain_instance(sample_text, predict_proba, num_features=10, top_labels=5)

print("\nLIME Analysis:")
exp.show_in_notebook()

print("\nModel Interpretability Report:")
print("1. SHAP Analysis: Highlights the importance of each token in the model's predictions.")
print("2. LIME Analysis: Provides local explanations for specific predictions.")
print("3. Recommendations: Add more diverse training data, fine-tune on overlapping entities, or experiment with different architectures.")